In [3]:
# lstm autoencoder recreate sequence
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

from get_data import GetData
from data_loader import DataLoader
from pre_processing import PreProcessing

import pywt

import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib qt

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [4]:
configs = json.load(open('config-test.json', 'r'))
configs

{'data': {'symbol': 'AMZN',
  'columns': ['Close'],
  'start': '2008-04-01',
  'end': '2019-10-01',
  'sequence_length': 5,
  'train_test_split': 0.85,
  'save_dir': 'data',
  'correlates_to': ['NOV',
   'EQIX',
   'CTAS',
   'CRM',
   'BSX',
   'IT',
   'V',
   'MSI',
   'CLX',
   'WAT']},
 'preprocessing': {'denoise': {'wavelet': 'haar', 'thr_mode': 'garotte'},
  'filename': 'preprocessed.csv',
  'save_dir': 'preprocessing'},
 'training': {'epochs': 2, 'batch_size': 32},
 'model': {'loss': 'mae',
  'optimizer': 'adam',
  'save_dir': 'model',
  'layers': [{'type': 'lstm',
    'neurons': 100,
    'activation': 'relu',
    'input_timesteps': 4,
    'input_dim': 1,
    'return_sequences': False},
   {'type': 'repeat_vector', 'output_shape': 1},
   {'type': 'lstm',
    'neurons': 100,
    'activation': 'relu',
    'return_sequences': True},
   {'type': 'time_distributed', 'layer': {'type': 'dense', 'neurons': 1}}]}}

In [5]:
sequence = pd.read_csv('./data/GOOGL.csv').Close[:]
sequence = (sequence - sequence.mean()) / (sequence.max() - sequence.min())

In [6]:
# denoise with haar wavelet
cA, cD = pywt.dwt(sequence, 'haar')
thr = np.std(sequence)/23
cA_shrinked = pywt.threshold(cA, thr, mode='garotte')
cD_shrinked = pywt.threshold(cD, thr, mode='garotte')
denoised = pywt.idwt(cA_shrinked, cD_shrinked, 'haar')
if len(denoised) > len(sequence):
    denoised = denoised[:-1]
denoised[:200]

array([-0.27580048, -0.27580048, -0.27691624, -0.27691624, -0.27296385,
       -0.27296385, -0.27540138, -0.27540138, -0.2805853 , -0.2805853 ,
       -0.28213873, -0.28213873, -0.27912437, -0.24778291, -0.24116879,
       -0.24116879, -0.24186848, -0.24186848, -0.24044119, -0.24044119,
       -0.23258968, -0.23258968, -0.22365772, -0.22365772, -0.22217863,
       -0.22217863, -0.22631101, -0.22631101, -0.22714174, -0.22714174,
       -0.22689274, -0.22689274, -0.22651278, -0.22651278, -0.22757536,
       -0.22757536, -0.23973935, -0.23973935, -0.23843651, -0.23843651,
       -0.2286229 , -0.2286229 , -0.22657074, -0.22657074, -0.23113865,
       -0.23113865, -0.2281807 , -0.2281807 , -0.23703709, -0.23703709,
       -0.24001838, -0.24001838, -0.23010832, -0.23010832, -0.23278716,
       -0.23278716, -0.23819828, -0.23819828, -0.24230203, -0.24230203,
       -0.24395248, -0.24395248, -0.24938876, -0.24938876, -0.24782637,
       -0.24782637, -0.24371855, -0.24371855, -0.24046265, -0.24

In [7]:
# reshape input into [samples, timesteps, features]
timesteps = 5
features = 1

X = []
i = 0
while i < len(denoised) - timesteps:
    t = []
    for j in range(timesteps):
        t.append(denoised[i+j])
    X.append(t)
    i += timesteps
    
X = np.array(X)
X = X.reshape((X.shape[0], X.shape[1], 1))
denoised = X

denoised[:2,:,0]

array([[-0.27580048, -0.27580048, -0.27691624, -0.27691624, -0.27296385],
       [-0.27296385, -0.27540138, -0.27540138, -0.2805853 , -0.2805853 ]])

In [45]:
#denoised = denoised[500:600]
denoised = denoised[10:110]
timesteps = len(denoised)
features = 1

denoised = np.reshape(denoised, (1, denoised.shape[0], 1))
denoised

array([[[-0.26683507],
        [-0.26683507],
        [-0.26234888],
        [-0.26234888],
        [-0.25813739],
        [-0.25813739],
        [-0.25722394],
        [-0.25722394],
        [-0.25047392],
        [-0.25047392],
        [-0.23947905],
        [-0.23947905],
        [-0.23639057],
        [-0.23639057],
        [-0.23280072],
        [-0.23280072],
        [-0.23020527],
        [-0.23020527],
        [-0.23913139],
        [-0.23913139],
        [-0.23925266],
        [-0.23925266],
        [-0.23924457],
        [-0.23924457],
        [-0.23808035],
        [-0.23808035],
        [-0.24446729],
        [-0.24446729],
        [-0.24694921],
        [-0.24694921],
        [-0.23718291],
        [-0.23718291],
        [-0.23293009],
        [-0.23293009],
        [-0.23243688],
        [-0.23243688],
        [-0.23417523],
        [-0.23417523],
        [-0.24099899],
        [-0.24099899],
        [-0.24516256],
        [-0.24516256],
        [-0.23909905],
        [-0

In [9]:
# define model
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(timesteps, features), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(RepeatVector(timesteps))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 128)           66560     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 10, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 64)            33024     
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 128)           98816     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 1)             129       
Total params: 247,937
Trainable params: 247,937
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(denoised, denoised, epochs=2000, batch_size=timesteps, verbose=1)

Epoch 1/2000
1/1 [==============================] - 2s 2s/step - loss: 0.0592
Epoch 2/2000
1/1 [==============================] - 0s 184ms/step - loss: 0.0554
Epoch 3/2000
1/1 [==============================] - 0s 198ms/step - loss: 0.0522
Epoch 4/2000
1/1 [==============================] - 0s 185ms/step - loss: 0.0480
Epoch 5/2000
1/1 [==============================] - 0s 184ms/step - loss: 0.0432
Epoch 6/2000
1/1 [==============================] - 0s 185ms/step - loss: 0.0378
Epoch 7/2000
1/1 [==============================] - 0s 186ms/step - loss: 0.0317
Epoch 8/2000
1/1 [==============================] - 0s 181ms/step - loss: 0.0250
Epoch 9/2000
1/1 [==============================] - 0s 186ms/step - loss: 0.0175
Epoch 10/2000
1/1 [==============================] - 0s 182ms/step - loss: 0.0099
Epoch 11/2000
1/1 [==============================] - 0s 196ms/step - loss: 0.0035
Epoch 12/2000
1/1 [==============================] - 0s 201ms/step - loss: 0.0022
Epoch 13/2000
1/1 [=========

Epoch 99/2000
1/1 [==============================] - 0s 186ms/step - loss: 6.9825e-04
Epoch 100/2000
1/1 [==============================] - 0s 180ms/step - loss: 6.9085e-04
Epoch 101/2000
1/1 [==============================] - 0s 181ms/step - loss: 6.8379e-04
Epoch 102/2000
1/1 [==============================] - 0s 184ms/step - loss: 6.7685e-04
Epoch 103/2000
1/1 [==============================] - 0s 180ms/step - loss: 6.6980e-04
Epoch 104/2000
1/1 [==============================] - 0s 185ms/step - loss: 6.6253e-04
Epoch 105/2000
1/1 [==============================] - 0s 181ms/step - loss: 6.5523e-04
Epoch 106/2000
1/1 [==============================] - 0s 184ms/step - loss: 6.4807e-04
Epoch 107/2000
1/1 [==============================] - 0s 185ms/step - loss: 6.4110e-04
Epoch 108/2000
1/1 [==============================] - 0s 191ms/step - loss: 6.3415e-04
Epoch 109/2000
1/1 [==============================] - 0s 184ms/step - loss: 6.2709e-04
Epoch 110/2000
1/1 [========================

Epoch 194/2000
1/1 [==============================] - 0s 179ms/step - loss: 1.5986e-04
Epoch 195/2000
1/1 [==============================] - 0s 180ms/step - loss: 1.5809e-04
Epoch 196/2000
1/1 [==============================] - 0s 182ms/step - loss: 1.5637e-04
Epoch 197/2000
1/1 [==============================] - 0s 179ms/step - loss: 1.5467e-04
Epoch 198/2000
1/1 [==============================] - 0s 182ms/step - loss: 1.5297e-04
Epoch 199/2000
1/1 [==============================] - 0s 186ms/step - loss: 1.5131e-04
Epoch 200/2000
1/1 [==============================] - 0s 183ms/step - loss: 1.4970e-04
Epoch 201/2000
1/1 [==============================] - 0s 181ms/step - loss: 1.4808e-04
Epoch 202/2000
1/1 [==============================] - 0s 182ms/step - loss: 1.4645e-04
Epoch 203/2000
1/1 [==============================] - 0s 181ms/step - loss: 1.4484e-04
Epoch 204/2000
1/1 [==============================] - 0s 187ms/step - loss: 1.4326e-04
Epoch 205/2000
1/1 [=======================

1/1 [==============================] - 0s 242ms/step - loss: 6.9346e-05
Epoch 289/2000
1/1 [==============================] - 0s 243ms/step - loss: 6.8728e-05
Epoch 290/2000
1/1 [==============================] - 0s 232ms/step - loss: 6.8168e-05
Epoch 291/2000
1/1 [==============================] - 0s 257ms/step - loss: 6.7606e-05
Epoch 292/2000
1/1 [==============================] - 0s 267ms/step - loss: 6.7010e-05
Epoch 293/2000
1/1 [==============================] - 0s 354ms/step - loss: 6.6405e-05
Epoch 294/2000
1/1 [==============================] - 0s 278ms/step - loss: 6.5826e-05
Epoch 295/2000
1/1 [==============================] - 0s 296ms/step - loss: 6.5275e-05
Epoch 296/2000
1/1 [==============================] - 0s 251ms/step - loss: 6.4725e-05
Epoch 297/2000
1/1 [==============================] - 0s 244ms/step - loss: 6.4165e-05
Epoch 298/2000
1/1 [==============================] - 0s 318ms/step - loss: 6.3593e-05
Epoch 299/2000
1/1 [==============================] - 0s 3

Epoch 383/2000
1/1 [==============================] - 0s 210ms/step - loss: 3.1427e-05
Epoch 384/2000
1/1 [==============================] - 0s 197ms/step - loss: 3.1105e-05
Epoch 385/2000
1/1 [==============================] - 0s 202ms/step - loss: 3.0718e-05
Epoch 386/2000
1/1 [==============================] - 0s 211ms/step - loss: 2.9920e-05
Epoch 387/2000
1/1 [==============================] - 0s 185ms/step - loss: 2.9233e-05
Epoch 388/2000
1/1 [==============================] - 0s 195ms/step - loss: 2.8875e-05
Epoch 389/2000
1/1 [==============================] - 0s 206ms/step - loss: 2.8823e-05
Epoch 390/2000
1/1 [==============================] - 0s 197ms/step - loss: 2.8933e-05
Epoch 391/2000
1/1 [==============================] - 0s 208ms/step - loss: 2.9020e-05
Epoch 392/2000
1/1 [==============================] - 0s 203ms/step - loss: 2.8927e-05
Epoch 393/2000
1/1 [==============================] - 0s 201ms/step - loss: 2.8284e-05
Epoch 394/2000
1/1 [=======================

...

Epoch 1605/2000
1/1 [==============================] - 0s 183ms/step - loss: 8.3863e-06
Epoch 1606/2000
1/1 [==============================] - 0s 186ms/step - loss: 8.5797e-06
Epoch 1607/2000
1/1 [==============================] - 0s 191ms/step - loss: 8.8394e-06
Epoch 1608/2000
1/1 [==============================] - 0s 181ms/step - loss: 9.1783e-06
Epoch 1609/2000
1/1 [==============================] - 0s 182ms/step - loss: 9.5472e-06
Epoch 1610/2000
1/1 [==============================] - 0s 180ms/step - loss: 9.7069e-06
Epoch 1611/2000
1/1 [==============================] - 0s 183ms/step - loss: 9.8011e-06
Epoch 1612/2000
1/1 [==============================] - 0s 184ms/step - loss: 9.6127e-06
Epoch 1613/2000
1/1 [==============================] - 0s 182ms/step - loss: 9.2935e-06
Epoch 1614/2000
1/1 [==============================] - 0s 181ms/step - loss: 8.9036e-06
Epoch 1615/2000
1/1 [==============================] - 0s 179ms/step - loss: 8.4880e-06
Epoch 1616/2000
1/1 [===========

1/1 [==============================] - 0s 181ms/step - loss: 8.1139e-06
Epoch 1699/2000
1/1 [==============================] - 0s 179ms/step - loss: 8.2004e-06
Epoch 1700/2000
1/1 [==============================] - 0s 181ms/step - loss: 8.2403e-06
Epoch 1701/2000
1/1 [==============================] - 0s 182ms/step - loss: 8.2278e-06
Epoch 1702/2000
1/1 [==============================] - 0s 180ms/step - loss: 8.2160e-06
Epoch 1703/2000
1/1 [==============================] - 0s 180ms/step - loss: 8.1176e-06
Epoch 1704/2000
1/1 [==============================] - 0s 181ms/step - loss: 8.0782e-06
Epoch 1705/2000
1/1 [==============================] - 0s 183ms/step - loss: 8.0068e-06
Epoch 1706/2000
1/1 [==============================] - 0s 183ms/step - loss: 8.0237e-06
Epoch 1707/2000
1/1 [==============================] - 0s 183ms/step - loss: 8.1598e-06
Epoch 1708/2000
1/1 [==============================] - 0s 181ms/step - loss: 8.3511e-06
Epoch 1709/2000
1/1 [===========================

Epoch 1792/2000
1/1 [==============================] - 0s 201ms/step - loss: 7.4764e-06
Epoch 1793/2000
1/1 [==============================] - 0s 192ms/step - loss: 7.3584e-06
Epoch 1794/2000
1/1 [==============================] - 0s 183ms/step - loss: 7.6643e-06
Epoch 1795/2000
1/1 [==============================] - 0s 201ms/step - loss: 7.6021e-06
Epoch 1796/2000
1/1 [==============================] - 0s 187ms/step - loss: 7.3033e-06
Epoch 1797/2000
1/1 [==============================] - 0s 200ms/step - loss: 7.4403e-06
Epoch 1798/2000
1/1 [==============================] - 0s 192ms/step - loss: 7.4401e-06
Epoch 1799/2000
1/1 [==============================] - 0s 192ms/step - loss: 7.3045e-06
Epoch 1800/2000
1/1 [==============================] - 0s 194ms/step - loss: 7.2914e-06
Epoch 1801/2000
1/1 [==============================] - 0s 189ms/step - loss: 7.3751e-06
Epoch 1802/2000
1/1 [==============================] - 0s 202ms/step - loss: 7.3124e-06
Epoch 1803/2000
1/1 [===========

1/1 [==============================] - 0s 181ms/step - loss: 6.3788e-06
Epoch 1886/2000
1/1 [==============================] - 0s 181ms/step - loss: 6.2978e-06
Epoch 1887/2000
1/1 [==============================] - 0s 181ms/step - loss: 6.3239e-06
Epoch 1888/2000
1/1 [==============================] - 0s 178ms/step - loss: 6.4761e-06
Epoch 1889/2000
1/1 [==============================] - 0s 179ms/step - loss: 6.7036e-06
Epoch 1890/2000
1/1 [==============================] - 0s 184ms/step - loss: 6.7143e-06
Epoch 1891/2000
1/1 [==============================] - 0s 179ms/step - loss: 6.5221e-06
Epoch 1892/2000
1/1 [==============================] - 0s 183ms/step - loss: 6.3161e-06
Epoch 1893/2000
1/1 [==============================] - 0s 183ms/step - loss: 6.2426e-06
Epoch 1894/2000
1/1 [==============================] - 0s 181ms/step - loss: 6.2459e-06
Epoch 1895/2000
1/1 [==============================] - 0s 182ms/step - loss: 6.2373e-06
Epoch 1896/2000
1/1 [===========================

Epoch 1979/2000
1/1 [==============================] - 0s 195ms/step - loss: 6.2103e-06
Epoch 1980/2000
1/1 [==============================] - 0s 192ms/step - loss: 6.2365e-06
Epoch 1981/2000
1/1 [==============================] - 0s 193ms/step - loss: 6.2484e-06
Epoch 1982/2000
1/1 [==============================] - 0s 198ms/step - loss: 6.1413e-06
Epoch 1983/2000
1/1 [==============================] - 0s 187ms/step - loss: 6.0575e-06
Epoch 1984/2000
1/1 [==============================] - 0s 196ms/step - loss: 6.0276e-06
Epoch 1985/2000
1/1 [==============================] - 0s 193ms/step - loss: 6.0275e-06
Epoch 1986/2000
1/1 [==============================] - 0s 184ms/step - loss: 5.9813e-06
Epoch 1987/2000
1/1 [==============================] - 0s 183ms/step - loss: 5.9295e-06
Epoch 1988/2000
1/1 [==============================] - 0s 185ms/step - loss: 5.9387e-06
Epoch 1989/2000
1/1 [==============================] - 0s 200ms/step - loss: 5.9571e-06
Epoch 1990/2000
1/1 [===========

In [72]:
from keras.models import load_model

model = load_model('./model-5timesteps-2000epochs.h5')

In [73]:
# demonstrate recreation
yhat = model.predict(denoised, verbose=0)
print(len(yhat.flatten()), len(denoised.flatten()))

2815 2815


In [74]:
plt.plot(denoised.flatten(), color='red', label='Sequence denoised')
plt.plot(yhat.flatten(), color='blue', label='Sequence reconstructed')
plt.legend()
plt.show()

In [76]:
reconstructed = yhat.flatten()

dataframe = pd.DataFrame({'Close': reconstructed})
dataframe.to_csv('../LSTM-Neural-Network-for-Time-Series-Prediction/data/R.csv', index=False)